# Generate Output
This notebook will read in the metadata from the [Arquin Spreadsheet](https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431) and product an output for ingestion into Omeka 

In [1]:
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

## import libraries

In [2]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime

In [8]:
%run functions.ipynb

## Load the spreadsheet information

In [28]:
# if HttpAccessTokenRefreshError, close everything and start over
sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

## This is testing to figure out how to get all the data where it needs to go!

### Get the list of Identifiers by Box Number

In [5]:
# get the list of boxes containing metadata

box_list = []
box_list_sheets = s.sheets.titles()

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))

In [ ]:
box_list

In [13]:
x = return_box(box_list[0])

In [49]:
# concepts[concepts['join_concept'].notna()]
x

,identifier,media,title,subject,description,creator,publisher,date,rights,language,relation,format,type,coverage,spatial
0,A-1 02,A-1_Arquin_001_0002.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi..."
1,A-1 03,A-1_Arquin_001_0003.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi..."
2,A-1 04,A-1_Arquin_001_0004.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi..."
3,A-1 05,A-1_Arquin_001_0005.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways)",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi..."
4,A-1 06,A-1_Arquin_001_0006.jpg,"Ostend Gate, The Ezelpoort","water gates, arch bridges, canals (waterways),...",Ostend Gate (Donkey's Gate) - 13th Century.,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,51.2093/ 3.2247,"Bruges (inhabited place), West Flanders (provi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,A-1 264,A-1_Arquin_001_0264.jpg,Church of Our Lady of Victories at the Sablon ...,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R..."
235,A-1 265,A-1_Arquin_001_0265.jpg,Church of Our Lady of Victories at the Sablon ...,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R..."
236,A-1 266,A-1_Arquin_001_0266.jpg,Church of Our Lady of Victories at the Sablon,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R..."
237,A-1 267,A-1_Arquin_001_0267.jpg,Church of Our Lady of Victories at the Sablon,"churches (buildings), architectural sculpture",Notre Dame Du Sablon,"Florence Arquin, photographer",Florida Atlantic University,NaN,NaN,NaN,NaN,35 mm kodachrome slide,still image,"50.8503° N, 4.3517° E","Brussels (inhabited place), Brussels-Capital R..."


### Get the title translations
from Box 'unique_titles'

In [29]:
title_translations = s.find('unique_titles').to_frame()

In [30]:
title_translations

,Index,Title,Spanish (from Google),Translation ES,EmptyColumn,Portuguese(from Google),Translation PT
0,1,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,NaN,"OSTEND GATE, O EZELPOORT",NaN
1,2,Street in Bruges,Calle en Brujas,NaN,NaN,NaN,NaN
2,3,Gruuthuse Palace,Palacio gruuthuse,NaN,NaN,NaN,NaN
3,4,Bonifacius Bridge,Puente de Bonifacio,NaN,NaN,NaN,NaN
4,5,Architecture in Bruges,Arquitectura en Brujas,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7171,7172,Market. Selling skeins of wool and woven woole...,Mercado. Vender madejas de lana y chales tejid...,NaN,NaN,NaN,NaN
7172,7173,Market. Dyed wool section.,Mercado. Sección de lana teñida.,NaN,NaN,NaN,NaN
7173,7174,Child and skeins of wool.,Niño y madejas de lana.,NaN,NaN,NaN,NaN
7174,7175,Market. Detail of dyed wool section.,Mercado. Detalle de la sección de lana teñida.,NaN,NaN,NaN,NaN


In [41]:
box_with_trans = x.merge(title_translations, left_on='title', right_on='Title')

In [48]:
box_with_trans[['identifier', 'media', 'title'
    ,'Spanish (from Google)' 
    ,'Translation ES', 'Portuguese(from Google)', 'Translation PT'
]]

,identifier,media,title,Spanish (from Google),Translation ES,Portuguese(from Google),Translation PT
0,A-1 04,A-1_Arquin_001_0004.jpg,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN
1,A-1 06,A-1_Arquin_001_0006.jpg,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN
2,A-1 07,A-1_Arquin_001_0007.jpg,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN
3,A-1 08,A-1_Arquin_001_0008.jpg,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN
4,A-1 09,A-1_Arquin_001_0009.jpg,"Ostend Gate, The Ezelpoort","Ostend Gate, el Ezelpoort",NaN,"OSTEND GATE, O EZELPOORT",NaN
...,...,...,...,...,...,...,...
201,A-1 261,A-1_Arquin_001_0261.jpg,Church of Our Lady of Victories at the Sablon ...,Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN
202,A-1 262,A-1_Arquin_001_0262.jpg,Church of Our Lady of Victories at the Sablon ...,Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN
203,A-1 263,A-1_Arquin_001_0263.jpg,Church of Our Lady of Victories at the Sablon ...,Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN
204,A-1 264,A-1_Arquin_001_0264.jpg,Church of Our Lady of Victories at the Sablon ...,Iglesia de Nuestra Señora de las Victorias en ...,NaN,NaN,NaN
